<a href="https://colab.research.google.com/github/mohsenMahmoodzadeh/Image-Caption-classification-with-tensorflow-Keras/blob/master/Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Classification with Keras

In this notebook, we build a keras model to classify a text dataset with 2658 train and 2658 test senetences. 

We use Tensorflow and Keras for implementation of our model. 

We also use scikit-learn library for some ancillary affairs such as confusion matrix, accuracy score, classification report, etc.

In [1]:
import tensorflow as tf
import tensorflow.keras as K
import random
import numpy as np
import pandas as pd
import h5py
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LSTM, Dropout, Activation, Embedding, Bidirectional
from tensorflow.keras import layers
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import KFold, train_test_split
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D,Activation,MaxPool2D,Dropout,Dense,Flatten,Input,BatchNormalization
from tensorflow.keras.models import Sequential,load_model,Model
from matplotlib import pyplot as plt
from collections import Counter
import os
import cv2
import warnings
warnings.filterwarnings('ignore','FutureWarning')

In [2]:
label_map = {
    0 : 'aeroplane', 1 : 'bicycle', 2 : 'bird', 3 : 'boat', 
    4 : 'bus', 5 : 'car', 6 : 'cat', 7 : 'chair', 8 : 'cow',
    9 : 'diningtable', 10 : 'dog', 11 : 'horse', 12 : 'motorbike',
    13 : 'person', 14 : 'pottedplant', 15 : 'sheep', 16 : 'sofa',
    17 : 'train', 18 : 'tvmonitor'
  }

In [3]:
CATEGORIES = list(label_map.values())

In [4]:
from google.colab import drive
drive.mount("/content/drive")
# !cp '/content/glove.6B.zip' "drive/My Drive/glove.6B.zip"

Mounted at /content/drive


In [5]:
def read_file(filename):
    f = open(filename, 'r')
    lines = f.readlines()
    return lines

In [6]:
def prepare_train_data(TRAIN_DATA_DIR, CATEGORIES):
    train_data = []
    path = ""
    for category in CATEGORIES:
        temp = []
        path = os.path.join(TRAIN_DATA_DIR, category)
        class_num = CATEGORIES.index(category)
        for filename in os.listdir(path):
            file_path = os.path.join(path, filename)
            try:
                lines = read_file(file_path)
                for line in lines:
                    temp.append(line)

            except Exception as e:
                print(str(e))
        
        for line in temp:
            train_data.append((line, class_num))
    
    return train_data

In [7]:
def prepare_test_data(TEST_DATA_DIR, CATEGORIES):
    test_data = []
    path = ""
    for category in CATEGORIES:
        temp = []
        path = os.path.join(TRAIN_DATA_DIR, category)
        class_num = CATEGORIES.index(category)
        for filename in os.listdir(path):
            file_path = os.path.join(path, filename)
            file = file_path
            try:
                lines = read_file(file)
                for line in lines:
                    temp.append(line)

            except Exception as e:
                print(str(e))
        
        for line in temp:
            test_data.append((line, class_num))
    return test_data

In [8]:
TRAIN_DATA_DIR = "/content/drive/MyDrive/dataset/train/sentences/"
TEST_DATA_DIR = "/content/drive/MyDrive/dataset/test/sentences/"
CATEGORIES = ["aeroplane", "bicycle", "bird", "boat", "bus", "car", "cat", "chair", "cow", "diningtable", "dog", "horse", 
              "motorbike", "person", "pottedplant", "sheep","sofa", "train", "tvmonitor"]

In [9]:
training_data = prepare_train_data(TRAIN_DATA_DIR, CATEGORIES)
test_data = prepare_test_data(TEST_DATA_DIR, CATEGORIES)

In [ ]:
# !pip install numpy requests nlpaug
# import nlpaug.augmenter.word as naw
# aug = naw.WordEmbsAug(model_type='glove', model_path="/content/glove.6B.50d.txt", action="insert")
# aug_training_data = [(aug.augment(training_data[i][0]), training_data[i][1]) for i in range(len(training_data))]

In [10]:
random.shuffle(training_data)
# random.shuffle(aug_training_data)
random.shuffle(test_data)

In [11]:
def extract_X_and_y(data):
  data_dict = {}
  X = []
  y = []
  for example in data:
    X.append(example[0])
    y.append(example[1])
  data_dict['X'] = X
  data_dict['y'] = y
  return data_dict

In [12]:
# training_dict = extract_X_and_y(aug_training_data)
training_dict = extract_X_and_y(training_data)
# X = training_dict['X']
# y = training_dict['y']
X_train = training_dict['X']
y_train = training_dict['y']

# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=0)

In [13]:
test_dict = extract_X_and_y(test_data)
X_test = test_dict['X']
y_test = test_dict['y']

In [ ]:
# def clean_data(text):  
#   word_tokens= text.lower().split()
#   le=WordNetLemmatizer()
#   stop_words= set(stopwords.words("english"))     
#   word_tokens= [le.lemmatize(w) for w in word_tokens if not w in stop_words]
  
#   cleaned_data=" ".join(word_tokens)
#   return cleaned_data

In [14]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
# cleaned_X_train = []
# for tr_example in X_train:
#   cleaned_data = clean_data(tr_example)
#   cleaned_X_train.append(cleaned_data)
#   # tokenizer.fit_on_texts(tr_example.strip())

In [ ]:
# cleaned_X_val = []
# for val_example in X_val:
#   cleaned_data = clean_data(tr_example)
#   cleaned_X_val.append(cleaned_data)

In [ ]:
# cleaned_X_test = []
# for test_example in X_test:
#   cleaned_data = clean_data(test_example)
#   cleaned_X_test.append(cleaned_data)

In [15]:
vocab_size = 2304 
oov_token = '<OOV>' #OOV = Out of Vocabulary
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_token)
maxlen = 100
padding_type = 'post'

In [16]:
def remove_stopwords(data):
  STOPWORDS= set(stopwords.words("english"))     
  no_sw_data = []
  for example in data:
      for word in STOPWORDS:
          token = ' ' + word + ' '
          example = example.replace(token, ' ')
          example = example.replace(' ', ' ')
      no_sw_data.append(example)
  return no_sw_data

In [17]:
X_train = remove_stopwords(X_train)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(X_train, maxlen=maxlen, padding=padding_type)
y_train_cat = tf.keras.utils.to_categorical(y_train)

In [ ]:
# X_val = remove_stopwords(X_val)
# tokenizer.fit_on_texts(X_val)
# X_val = tokenizer.texts_to_sequences(X_val)
# X_val = pad_sequences(X_val, maxlen=maxlen, padding=padding_type)
# y_val_cat = tf.keras.utils.to_categorical(y_val)

In [18]:
X_test = remove_stopwords(X_test)
tokenizer.fit_on_texts(X_test)
X_test = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(X_test, maxlen=maxlen, padding=padding_type)
y_test_cat = tf.keras.utils.to_categorical(y_test)

In [19]:
import zipfile
zip_ref = zipfile.ZipFile('/content/drive/MyDrive/glove.6B.zip', 'r')
zip_ref.extractall('/content/')
zip_ref.close()

In [ ]:
# import gensim
# sentences= cleaned_X_train + cleaned_X_val + cleaned_X_test
# w2v_model=gensim.models.Word2Vec(sentences= sentences, size=300, window=10, min_count=1)

In [ ]:
# w2v_model.train(sentences,epochs=10,total_examples=len(sentences))

In [ ]:
# vocab=w2v_model.wv.vocab
# print("The total number of words are : ",len(vocab))

In [ ]:
# vocab=list(vocab.keys())

In [ ]:
# word_vec_dict={}
# for word in vocab:
#   word_vec_dict[word]=w2v_model.wv.get_vector(word)
# print("The no of key-value pairs : ",len(word_vec_dict))

In [ ]:
# tokenizer.fit_on_texts(sentences)
# vocab_size = len(tokenizer.word_index) + 1

In [ ]:
# tokenizer.fit_on_texts(cleaned_X_rtain)
# X_train = tokenizer.texts_to_sequences(cleaned_X_rtain)
# X_train = pad_sequences(X_train, maxlen=maxlen, padding=padding_type)
# y_train_cat = tf.keras.utils.to_categorical(y_train)

In [ ]:
# tokenizer.fit_on_texts(cleaned_X_val)
# X_val = tokenizer.texts_to_sequences(cleaned_X_val)
# X_val = pad_sequences(X_val, maxlen=maxlen, padding=padding_type)
# y_val_cat = tf.keras.utils.to_categorical(y_val)

In [ ]:
# tokenizer.fit_on_texts(cleaned_X_test)
# X_test = tokenizer.texts_to_sequences(cleaned_X_test)
# X_test = pad_sequences(X_test, maxlen=maxlen, padding=padding_type)
# y_test_cat = tf.keras.utils.to_categorical(y_test)

In [20]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

In [21]:
embedding_dim = 50
embedding_matrix = create_embedding_matrix('/content/glove.6B.50d.txt',tokenizer.word_index, embedding_dim)

In [22]:
nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
nonzero_elements / vocab_size

0.9800347222222222

In [23]:
def create_model(vocab_size, embedding_dim, maxlen):
  num_classes= 19
  model = Sequential()
  
  model.add(layers.Embedding(vocab_size, embedding_dim, 
                            weights=[embedding_matrix], 
                            input_length=maxlen, 
                            trainable=True
                            ))
  model.add(layers.Conv1D(128, 5, activation='relu'))
  model.add(layers.GlobalMaxPool1D())
  model.add(layers.Dropout(0.3))
  model.add(layers.Dense(20, activation='relu'))
  # model.add(layers.Dropout(0.1))
  model.add(layers.Dense(num_classes, activation='softmax'))

  model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  return model


In [ ]:
# plt.style.use('ggplot')

# def plot_history(history):
#     acc = history.history['acc']
#     val_acc = history.history['val_acc']
#     loss = history.history['loss']
#     val_loss = history.history['val_loss']
#     x = range(1, len(acc) + 1)

#     plt.figure(figsize=(12, 5))
#     plt.subplot(1, 2, 1)
#     plt.plot(x, acc, 'b', label='Training acc')
#     plt.plot(x, val_acc, 'r', label='Validation acc')
#     plt.title('Training and validation accuracy')
#     plt.legend()
#     plt.subplot(1, 2, 2)
#     plt.plot(x, loss, 'b', label='Training loss')
#     plt.plot(x, val_loss, 'r', label='Validation loss')
#     plt.title('Training and validation loss')
#     plt.legend()

In [ ]:
# print(vocab_size)

In [25]:
model = create_model(vocab_size, embedding_dim, maxlen)
batch_size = 32
epochs = 10
history = model.fit(X_train, y_train_cat,
                    epochs=epochs,
                    # validation_data=(X_val, y_val_cat),
                    validation_split=0.1,
                    batch_size=batch_size)

Epoch 1/10
75/75 [==============================] - 3s 26ms/step - loss: 2.7239 - accuracy: 0.1442 - val_loss: 2.3580 - val_accuracy: 0.2895
Epoch 2/10
75/75 [==============================] - 2s 22ms/step - loss: 2.0575 - accuracy: 0.3821 - val_loss: 1.7830 - val_accuracy: 0.4887
Epoch 3/10
75/75 [==============================] - 2s 22ms/step - loss: 1.5048 - accuracy: 0.5497 - val_loss: 1.4056 - val_accuracy: 0.5677
Epoch 4/10
75/75 [==============================] - 2s 22ms/step - loss: 1.1266 - accuracy: 0.6710 - val_loss: 1.2765 - val_accuracy: 0.6316
Epoch 5/10
75/75 [==============================] - 2s 22ms/step - loss: 0.8961 - accuracy: 0.7391 - val_loss: 1.1487 - val_accuracy: 0.6617
Epoch 6/10
75/75 [==============================] - 2s 23ms/step - loss: 0.7211 - accuracy: 0.7964 - val_loss: 1.0397 - val_accuracy: 0.6917
Epoch 7/10
75/75 [==============================] - 2s 24ms/step - loss: 0.6136 - accuracy: 0.8278 - val_loss: 1.0133 - val_accuracy: 0.7030
Epoch 8/10
75

In [26]:
predictions = model.predict(X_test)
y_pred = np.argmax(predictions, axis=1)

In [33]:
print('Confusion Matrix')
print(confusion_matrix(y_test, y_pred))

print()

print('Classification Report')
print(classification_report(y_test, 
                            y_pred, 
                            target_names=CATEGORIES))

Confusion Matrix
[[138   0   0   2   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0]
 [  0 137   0   0   0   0   0   1   0   1   0   0   0   1   0   0   0   0
    0]
 [  0   0 140   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0]
 [  1   0   0 138   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0]
 [  3   3   0   0 131   1   0   0   0   0   0   0   0   2   0   0   0   0
    0]
 [  5   3   0   0   6 117   0   1   1   1   0   0   4   2   0   0   0   0
    0]
 [  0   0   0   0   0   0 137   0   1   0   1   0   0   0   1   0   0   0
    0]
 [  0   0   0   0   0   0   1 125   0   5   0   1   0   1   2   0   3   0
    2]
 [  0   0   2   0   0   0   0   0 133   0   2   2   0   0   0   0   0   0
    0]
 [  0   0   0   0   0   0   0   5   0 120   1   0   0   5   3   0   3   0
    3]
 [  0   0   0   0   0   0   0   2   0   0 132   0   0   0   2   4   0   0
    0]
 [  0   0   0   0   0   0   0   0   0   0   0 136   0   4   0   0   0   0
    0]
 [  0   5  

In [30]:
loss, accuracy = model.evaluate(X_train, y_train_cat, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test_cat, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
# plot_history(history)

Training Accuracy: 0.9372
Testing Accuracy:  0.9372


In [ ]:
# kfold = KFold(n_splits=10, shuffle=True)
# inputs = np.concatenate((X_train, X_test), axis=0)
# targets = np.concatenate((y_train_cat, y_test_cat), axis=0)
# fold_no = 1
# acc_per_fold = []
# loss_per_fold = []
# learning_rate= 0.0001

In [ ]:
# for train, test in kfold.split(inputs, targets):

#   model = create_model(vocab_size, embedding_dim, maxlen)

#   model.compile(loss=categorical_crossentropy, optimizer=Adam(lr= learning_rate), metrics=['accuracy'])

#   print('------------------------------------------------------------------------')
#   print(f'Training for fold {fold_no} ...')

#   history = model.fit(inputs[train], targets[train],
#               batch_size=batch_size,
#               validation_data=(X_val, y_val_cat),
#               epochs=epochs)

#   scores = model.evaluate(inputs[test], targets[test], verbose=0)
#   print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
#   acc_per_fold.append(scores[1] * 100)
#   loss_per_fold.append(scores[0])

#   fold_no = fold_no + 1


# print('------------------------------------------------------------------------')
# print('Score per fold')
# for i in range(0, len(acc_per_fold)):
#   print('------------------------------------------------------------------------')
#   print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
# print('------------------------------------------------------------------------')
# print('Average scores for all folds:')
# print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
# print(f'> Loss: {np.mean(loss_per_fold)}')
# print('------------------------------------------------------------------------')

In [ ]:
# def cross_validate(X, y, K = 5, **kwargs):
#     scores = []
#     histories = []
#     for train, test in KFold(n_splits=K, shuffle=True).split(X,y):
        
#         model = create_model(vocab_size, embedding_dim, maxlen)
#         model.compile(loss=categorical_crossentropy, optimizer=Adam(learning_rate= learning_rate), metrics=['accuracy'])
        
#         histories.append(model.fit(X[train], y[train],
#                                    validation_data = (X_val, y_val_cat),
#                                    epochs=epochs,
#                                    batch_size= batch_size,
#                                    **kwargs).history)
        
#         scores.append(model.evaluate(X[test], y[test], verbose = 0))
#     print("average test loss: ", np.asarray(scores)[:,0].mean())
#     print("average test accuracy: ", np.asarray(scores)[:,1].mean())
    
#     return scores, histories

In [ ]:
# scores, histories = cross_validate(inputs, targets, K = 10)

## Resources

1. https://realpython.com/python-keras-text-classification/
2. https://stackoverflow.com/questions/50060241/how-to-use-glove-word-embeddings-file-on-google-colaboratory
3. https://nlp.stanford.edu/projects/glove/
4. https://www.machinecurve.com/index.php/2020/02/18/how-to-use-k-fold-cross-validation-with-keras/
5. https://stackoverflow.com/questions/45117295/what-is-the-relation-between-validation-data-and-validation-split-in-keras-fit
6. https://towardsdatascience.com/addressing-the-difference-between-keras-validation-split-and-sklearn-s-train-test-split-a3fb803b733
7. https://towardsdatascience.com/data-augmentation-library-for-text-9661736b13ff
8. https://github.com/makcedward/nlpaug
9. https://www.depends-on-the-definition.com/guide-to-word-vectors-with-gensim-and-keras/
10. https://code.google.com/archive/p/word2vec/